In [1]:
import pandas as pd
import sys
sys.path.append("c:/users/user/Downloads/film/Business-Driven-Oil-Price-Modeling/src")
from data_processing import (load_data,reshape_to_long_format,filter_by_year,resample_annual_avg,merge_datasets,filter_by_countries,handle_missing_values)

In [2]:
gdp =load_data("../data/API_NY.GDP.MKTP.KD.ZG_DS2_en_csv_v2_10065.csv", columns=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'] + list(map(str, range(1960, 2023 + 1))))
unemployment=load_data("../data/API_SL.UEM.TOTL.ZS_DS2_en_csv_v2_10162.csv",columns=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'] + list(map(str, range(1960, 2023 + 1))))
inflation=load_data("../data/API_FP.CPI.TOTL.ZG_DS2_en_csv_v2_10107.csv",columns=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'] + list(map(str, range(1960, 2023 + 1))))
brent_data =load_data("../data/Copy of BrentOilPrices.csv")

In [3]:
column_names=["Date","Price"]
brent_data.columns=column_names
brent_data=brent_data.set_index("Date",drop=True)
brent_data.index=pd.to_datetime(brent_data.index)

C:\Users\User\AppData\Local\Temp\ipykernel_7144\1953012716.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  brent_data.index=pd.to_datetime(brent_data.index)


In [4]:
brent_data.head()

,Price
Date,
1987-05-26,18.63
1987-05-27,18.60
1987-05-28,18.60
1987-05-29,18.58
1987-06-01,18.65


In [5]:
countries = ['China', 'United States', 'Saudi Arabia', 'Russia']

In [6]:
gdp_filtered = filter_by_countries(gdp, countries)
inflation_filtered = filter_by_countries(inflation, countries)
unemployment_filtered = filter_by_countries(unemployment, countries)

In [7]:
gdp_data_long = reshape_to_long_format(gdp_filtered, id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'])
unemployment_data_long = reshape_to_long_format(unemployment_filtered,id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'])
inflation_data_long = reshape_to_long_format(inflation_filtered,id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'])

In [8]:
gdp_data_filtered = filter_by_year(gdp_data_long, 1987, 2022)
unemployment_data_filtered = filter_by_year(unemployment_data_long,1987,2022)
inflation_data_filtered =filter_by_year(inflation_data_long,1987,2022)

In [9]:
gdp_data_filtered.head()

,Country Name,Country Code,Indicator Name,Indicator Code,Year,Value
81,China,CHN,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,1987,11.657428
82,Saudi Arabia,SAU,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,1987,-6.632517
83,United States,USA,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,1987,3.454630
84,China,CHN,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,1988,11.222595
85,Saudi Arabia,SAU,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,1988,13.109309


In [10]:
# Resample the data to annual frequency and calculate the mean for each year
annual_data = brent_data.resample('Y').mean()

# Create a new column 'Year' from the index after resetting it
annual_data = annual_data.reset_index()
annual_data['Year'] = annual_data['Date'].dt.year  # Assuming the index was originally set to 'Date'

# Select only the 'Year' column and the specified value column (e.g., 'Price')
brent_annual_data = annual_data[['Year',"Price"]]

C:\Users\User\AppData\Local\Temp\ipykernel_7144\3355053199.py:2: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  annual_data = brent_data.resample('Y').mean()


In [11]:
brent_annual_data.head()

,Year,Price
0,1987,18.525000
1,1988,14.905412
2,1989,18.228228
3,1990,23.761445
4,1991,20.041128


In [12]:
economic_data_combined = pd.concat([inflation_data_filtered, unemployment_data_filtered, gdp_data_filtered])
# merged_data = merge_datasets(economic_data_combined, brent_annual_data, on="Date", indicator_name="GDP")

In [13]:
economic_data_combined.head()

,Country Name,Country Code,Indicator Name,Indicator Code,Year,Value
81,China,CHN,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,1987,7.233836
82,Saudi Arabia,SAU,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,1987,-1.547117
83,United States,USA,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,1987,3.664563
84,China,CHN,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,1988,18.811818
85,Saudi Arabia,SAU,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,1988,0.907563


In [14]:
merged_data = merge_datasets(economic_data_combined, brent_annual_data)


In [15]:
final_data = handle_missing_values(merged_data)

c:\users/user/Downloads/film/Business-Driven-Oil-Price-Modeling/src\data_processing.py:128: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  return data.fillna(method=method)


In [19]:
final_data["Indicator Name"].value_counts()

Indicator Name
Inflation, consumer prices (annual %)                                  108
Unemployment, total (% of total labor force) (modeled ILO estimate)    108
GDP growth (annual %)                                                  108
Name: count, dtype: int64

In [20]:
final_data.to_csv("final_data.csv")